# 1. Información inicial

**Challenge Telecom X**

Análisis de evasión de clientes (Churn de clientes)

**Objetivo:**

Buscar, tratar y realizar un análisis exploratorio de los datos de la empresa. Es decir, todo el proceso ETL.



# 2. Extracción

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 2.1 Carga archivo json desde API

In [2]:
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"
response = requests.get(url)

telecom = response.json() 
telecom = pd.json_normalize(telecom, sep='_')
# Quita los prefijos: customer_, phone_, etc.
telecom.columns = [col.split('_')[-1] for col in telecom.columns]

df_telecom = pd.DataFrame(telecom)
df_telecom.head(5)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Monthly,Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


## 2.2 Revisa nulos

In [4]:
df_telecom.isna().sum()

customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Monthly             0
Total               0
dtype: int64

# 3. Transformación

## 3.1 Tipos de datos del conjunto

### Diccionario de datos

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente

In [5]:
df_telecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


## 3.2 Conversión de columnas

### 3.2.1 Convertir columna ```Churn``` en bool

In [6]:
df_telecom['Churn'] = df_telecom['Churn'].map({'Yes':True, 'No':False}).astype(bool)
df_telecom['Churn'].dtypes

dtype('bool')

### 3.2.2 Convertir columna ```SeniorCitizen``` en bool

In [12]:
df_telecom['SeniorCitizen'] = df_telecom['SeniorCitizen'].map({0:False, 1:True}).astype(bool)
df_telecom['SeniorCitizen'].dtypes

dtype('bool')

### 3.2.3 Convertir el resto de columnas con yes, no en bool
```Partner, Dependents, PhoneService, MultipleLines, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, PaperlessBilling```

In [18]:
cols_a_Bool = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineBackup', \
               'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
df_telecom[cols_a_Bool] = df_telecom[cols_a_Bool].map(lambda x: True if x == 'Yes' else False).astype(bool)

### 3.2.4 Convertir columna ```Total``` en float

In [33]:
df_telecom['Total'] = pd.to_numeric(df_telecom['Total'], errors='coerce')
df_telecom['Total'].dtypes

dtype('float64')

## 3.3 Revisa si hay duplicados

In [35]:
df_telecom[df_telecom.duplicated()]

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Monthly,Total


No hay duplicados

## 3.4 Creación columna ```Cuentas_diarias```

In [39]:
df_telecom['Cuentas_diarias'] = round(df_telecom['Monthly']/30, 2)
df_telecom[['Monthly', 'Cuentas_diarias']]

,Monthly,Cuentas_diarias
0,65.60,2.19
1,59.90,2.00
2,73.90,2.46
3,98.00,3.27
4,83.90,2.80
...,...,...
7262,55.15,1.84
7263,85.10,2.84
7264,50.30,1.68
7265,67.85,2.26


# 4. Carga y análisis

# 5. Informe final